# iSWAP Gate

*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
This tutorial will demonstrate how to generate the fideliy-optimized pulse for the iSWAP gate using Quanlse. The outline of this tutorial is as follows:
- Preparation
- Introduction
- Quanlse optimization
- Summary

In the **Introduction** section, we will give a brief introduction on iSWAP gate and construct the system Hamiltonian. Then we will demonstrate how to generate fideliy-optimized pulse sequence using Quanlse Cloud Service in the **Quanlse optimization** section.

## Preparation
After you have successfully installed Quanlse on your device, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# Import numpy and math constants
from numpy import round
from math import pi

# Import the Hamiltonian module
from Quanlse.Utils import Hamiltonian as qham

# Import related packages
from Quanlse.Utils.Operator import duff, number
from Quanlse.remoteOptimizer import remoteOptimizeISWAP

# Import tools for analysing results
from Quanlse.Utils.Tools import project

To use the Quanlse Cloud Service, we need to acquire a token to get access to the cloud.

In [ ]:
# Import Define class and set the token for cloud service
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ""

## Introduction
The iSWAP gate swaps the excitations between two qubits and adds a phase of $i$ \[1\].
The corresponding unitary matrix is:
$$
\rm iSWAP = \begin{pmatrix}
    1 & 0 & 0 & 0 \\
    0 & 0 & i & 0 \\
    0 & i & 0 & 0 \\
    0 & 0 & 0 & 1   
\end{pmatrix}. \\ 
$$
The iSWAP gate is a native gate in superconducting quantum computing platforms, because it can be realized directly using coupling between two superconducting qubits (XY interaction). In superconducting circuits, the iSWAP gate is implemented by tuning the qubits into resonance in a duration of time. By performing a rotating wave approximation, the two-qubit coupling Hamiltonian can be written as \[2\]:
$$
\hat H_{\rm couple} = g(e^{i\delta_{12}t}\hat{\sigma} ^+\hat{\sigma} ^-+e^{-i\delta_{12}t}\hat{\sigma} ^-\hat{\sigma} ^+),
$$
where $\delta_{12}=\omega_{q1}-\omega_{q2}$ is the detuning. By tuning the frequency of qubit 1 into resonance with qubit 2, the detuning $\delta_{12}$ reduces to $0$. The coupling Hamiltonian reduces to:
$$
\hat H_{\rm couple} = g(\hat{\sigma} ^+\hat{\sigma} ^-+\hat{\sigma} ^-\hat{\sigma} ^+),
$$
which swaps the excitations of the two qubits.

Moreover, two iSWAP gates and several single-qubit gates can be used to generate a CNOT gate. The generation of GHZ states utilizing the iSWAP gates has also been demonstrated \[2\].

## Quanlse optimization

To perform optimization using Quanlse, we need to specify the system Hamiltonian. We consider the three lowest energy levels of each qubit where the third level accounts for leakage. In the rotating frame, the system Hamiltonian of two coupled qubits is expressed as \[1\]:
$$
\hat H =(\omega_{q1}-\omega_{d1})(\hat a^\dagger_1 \hat a_1)+(\omega_{q2}-\omega_{d2})(\hat a^\dagger_2 \hat a_2)+ \frac{\alpha _1}{2}\hat a^\dagger_1 \hat a^\dagger_1 \hat a_1 \hat a_1 + \frac{\alpha_2}{2}\hat a^\dagger_2 \hat a^\dagger_2 \hat a_2 \hat a_2 +\frac{g}{2}(\hat a_1\hat a_2^\dagger+\hat a_1^\dagger \hat a_2)+\frac{A^z_1(t)}{2}\hat a^\dagger_1 \hat a_1,
$$
where $\hat a_i^\dagger$ ($\hat a_i$) is the creation (annihilation) operator of qubit $i$ ($i = 1,2$); $\omega _{qi}$ is the frequency of qubit $i$; $\alpha_i$ is the anharmonicity of qubit $i$; and $g$ is the coupling strength between the qubits. $A^z_1(t)$ is the waveform function that controls the flux of qubit 1 and tunes its frequency.

Quanlse defines the system Hamiltonian by specifying the sampling period, gate time and everything else necessary to set up the calculation. Quanlse provides the flexibility to customize simulation for practically any gate operations, particularly those involving parameter optimization. First, we define a couple of parameters to set the sampling period, the number of qubits, and the system dimension: 

In [ ]:
# Sampling period
dt = 1.0
# Number of qubits
qubits = 2
# System energy level
level = 3

We then define the transition frequencies, drive frequencies, and anharmonicities of the two qubits and the coupling strength in between:

In [ ]:
qubitArgs = {}
# Coupling strength, GHz
g = 0.0277 * (2 * pi)
# Transition frequency, GHz
qubit_freq0 = 5.805 * (2 * pi)
qubit_freq1 = 5.205 * (2 * pi)
# Microwave drive frequency, GHz
drive_freq0 = qubit_freq1
drive_freq1 = qubit_freq1
# Anharmonicity, GHz
anharm0 = -0.217 * (2 * pi)
anharm1 = -0.226 * (2 * pi)

Then, we can proceed to create an empty Hamiltonian dictionary using the `createHam()` function and add terms to it based on previously defined parameters:
$$
\hat H_{\rm couple} = \frac{g}{2}(\hat a_1\hat a_2^\dagger+\hat a_1^\dagger \hat a_2),
$$
$$
\hat H_{\rm ctrl} = \frac{A^z_1(t)}{2}\hat a^\dagger_1 \hat a_1,
$$
$$
\hat H_{\rm drift} = (\omega_{q1}-\omega_{d1})(\hat a^\dagger_1 \hat a_1)+(\omega_{q2}-\omega_{d2})(\hat a^\dagger_2 \hat a_2)+\frac{\alpha _1}{2}\hat a^\dagger_1 \hat a^\dagger_1 \hat a_1 \hat a_1 + \frac{\alpha_2}{2}\hat a^\dagger_2 \hat a^\dagger_2 \hat a_2 \hat a_2.
$$

In [ ]:
ham = qham.createHam(title="2q-3l", dt=dt, qubitNum=qubits, sysLevel=level)

# Add the detuning terms
detune0 = qubit_freq0 - drive_freq0
qham.addDrift(ham, name=f"q0-detuning", onQubits=0, matrices=number(level), amp=detune0)
detune1 = qubit_freq1 - drive_freq1
qham.addDrift(ham, name=f"q1-detuning", onQubits=1, matrices=number(level), amp=detune1)
# Add the anharmonicity terms
qham.addDrift(ham, name=f"q0-anharm", onQubits=0, matrices=duff(level), amp=0.5 * anharm0)
qham.addDrift(ham, name=f"q1-anharm", onQubits=1, matrices=duff(level), amp=0.5 * anharm1)
# Add the coupling term
qham.addCoupling(ham, name="coupling", onQubits=[0, 1], g=g / 2.0)
# Add the control term
qham.addControl(ham, name="q0-ctrlz", onQubits=0, matrices=number(level))

All useful information regarding the system Hamiltonian is stored in `ham`. We should set a boundary for amplitudes before launching the optimization. You might need to run the optimization several times and narrow down the boundary gradually.

In [ ]:
aBound = (-5, 5)

Now, we are ready to run the optimization. The `remoteOptimizeISWAP()` function takes five arguments. `tg` specifies the gate time. When `targetInfidelity` is reached, or the number of iterations exceeds `maxIter`, the optimization will terminate and return the minimum infidelity and the system Hamiltonian with optimized control terms. We can plot the pulses by calling `qham.plotWaves()` and get the unitary evolution by calling `qham.getUnitary()`. 

The gate infidelity for performance assessment throughout this tutorial is defined as ${\rm infid} = 1 - \frac{1}{d}\left|{\rm Tr}[U^\dagger_{\rm goal}P(U)]\right|$, where $d$ is the dimension of $U_{\rm goal}$ and $U$ is the unitary evolution. Note that $P(U)$ in particular describes the evolution projected to the computational subspace. We can first  run the optimization then calculate the projected evolution $P(U)$ :

In [ ]:
# Run the optimization
ham, infidelity = remoteOptimizeISWAP(ham, aBound=aBound, tg=40, maxIter=3, targetInfidelity=0.005)

# Print infidelity and the waveforms
print(f"minimum infidelity: {infidelity}")
qham.plotWaves(ham, ["q0-ctrlz"])

# Print the projected evolution
result = qham.simulate(ham)
print("The projected evolution P(U):\n", round(project(result["unitary"], qubits, level, 2), 2))

We can see that the drive strength of the pulse is around 3.8. We can zoom in and search for a better fidelity by narrowing down the boundary to (-4.0, -3.6).

In [ ]:
aBound = (-4.0, -3.6)
# Run the optimization
ham, infidelity = remoteOptimizeISWAP(ham, aBound, tg=40, maxIter=3, targetInfidelity=0.01)

# Print infidelity and the waveforms
print(f"minimum infidelity: {infidelity}")
qham.plotWaves(ham, ["q0-ctrlz"])

# Print the projected evolution 
result = qham.simulate(ham)
print("The projected evolution P(U):\n", round(project(result["unitary"], qubits, level, 2), 2))

If you wish to further reduce the infidelity, try setting the parameter `maxIter` to a larger number. With a set of reasonable parameters, Quanlse can always find the optimal waveforms.

## Summary
This tutorial introduces the pulse optimization of the iSWAP gate using Quanlse. After reading this tutorial, the users could follow this link [tutorial-iswap](https://github.com/baidu/Quanlse/tree/master/Tutorial/EN/tutorial-iswap.ipynb) to the GitHub page of this Jupyter Notebook document and run this program for themselves. The users are encouraged to try parameter values different from this tutorial to obtain the optimal result.

## References
\[1\] [Schuch, Norbert, and Jens Siewert. "Natural two-qubit gate for quantum computation using the XY interaction." *Physical Review A* 67.3 (2003): 032301.](https://link.aps.org/doi/10.1103/PhysRevA.67.032301)

\[2\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://doi.org/10.1063/1.5089550)

